In [31]:
import jisilu.jisilu_data as jisilu_data

data = jisilu_data.Jisilu().run()

ndata = data[(data['转债剩余占总市值比'] < 50) & (data['剩余时间'] > 0.3)].sort_values('双低', ascending=True).head(10)


登录成功


In [6]:
import akshare as ak

# 获取所有ETF的信息
etf_info = ak.fund_etf_category_sina(symbol="ETF基金")

# 打印结果
# print(etf_info)

In [7]:
etf_info[etf_info['代码'].str.contains('588000', na=False)]

,代码,名称,最新价,涨跌额,涨跌幅,买入,卖出,昨收,今开,最高,最低,成交量,成交额
457,sh588000,科创50ETF,1.122,0.041,3.793,1.122,1.123,1.081,1.082,1.128,1.082,7326389663,8150096563


In [1]:
import akshare as ak

# 获取所有A股股票的代码和名称
stock_info = ak.stock_info_a_code_name()

# 打印结果
print(stock_info)

  0%|          | 0/13 [00:00<?, ?it/s]

        code   name
0     000001   平安银行
1     000002  万  科Ａ
2     000004   国华网安
3     000006   深振业Ａ
4     000007    全新好
...      ...    ...
5361  920066    科拜尔
5362  920088   科力股份
5363  920099   瑞华技术
5364  920111    N聚星
5365  920118   太湖远大

[5366 rows x 2 columns]


In [ ]:
stock_info

In [12]:
import akshare as ak
import pandas as pd
from datetime import datetime, timedelta

def get_cb_turnover_rate(bond_code, days=5):
    """获取可转债近N日换手率
    
    Args:
        bond_code: 可转债代码，如 '127061'（不带市场标识）
        days: 获取天数，默认5天
    
    Returns:
        包含日期和换手率的字典列表
    """
    try:
        # 获取可转债历史数据
        df = ak.bond_zh_hs_cov_hist(symbol=bond_code)
        
        # 确保数据按日期降序排列（最新日期在前）
        df['日期'] = pd.to_datetime(df['日期'])
        df = df.sort_values('日期', ascending=False)
        
        # 选取最近N天的数据
        df = df.head(days)
        
        # 转换为列表格式返回
        result = []
        for _, row in df.iterrows():
            result.append({
                'date': row['日期'].strftime('%Y-%m-%d'),
                'turnover_rate': round(float(row['换手率']), 2)
            })
        
        return result
    
    except Exception as e:
        print(f"获取数据出错: {e}")
        return None

# 使用示例
if __name__ == "__main__":
    bond_code = '127061'  # 以美锦转债为例
    turnover_rates = get_cb_turnover_rate(bond_code)
    
    if turnover_rates:
        print(f"{bond_code} 最近5日换手率：")
        for item in turnover_rates:
            print(f"日期: {item['date']}, 换手率: {item['turnover_rate']}%")

获取数据出错: module 'akshare' has no attribute 'bond_zh_hs_cov_hist'


In [3]:
from init import xtdata

In [2]:
from miniqmt import DataAPI

def get_cb_turnover_rate(bond_code, days=5):
    """获取转债近N日换手率
    
    Args:
        bond_code: 转债代码，如 '123456'
        days: 获取天数，默认5天
    
    Returns:
        包含日期和换手率的字典列表
    """
    # 获取日线数据
    df = DataAPI.get_history_data(
        symbol=bond_code,
        period='1d',
        field_list=['date', 'turn'],
        count=days
    )
    
    # 转换为列表格式返回
    result = []
    for _, row in df.iterrows():
        result.append({
            'date': row['date'].strftime('%Y-%m-%d'),
            'turnover_rate': round(row['turn'], 2)
        })
    
    return result

# 使用示例
bond_code = '127061.SZ'  # 替换为实际的转债代码
turnover_rates = get_cb_turnover_rate(bond_code)
for item in turnover_rates:
    print(f"日期: {item['date']}, 换手率: {item['turnover_rate']}%")

KeyboardInterrupt: 

In [20]:
from xtquant import xtdata

df = xtdata.get_market_data(stock_list=["110092.SH"], period='1d')


In [23]:
import akshare as ak

bond_zh_hs_cov_min_df = ak.bond_zh_hs_cov_min(symbol="sz123124", period='1', adjust='', start_date="1979-09-01 09:32:00", end_date="2222-01-01 09:32:00")

In [27]:
import akshare as ak

bond_zh_hs_cov_min_df = ak.bond_zh_hs_cov_min(symbol="sz110092", period='5', adjust='hfq', start_date="2023-09-01 09:32:00", end_date="2023-09-06 09:32:00")


ValueError: Length mismatch: Expected axis has 0 elements, new values have 11 elements

In [26]:
bond_zh_hs_cov_min_df

,时间,开盘,收盘,最高,最低,涨跌幅,涨跌额,成交量,成交额,振幅,换手率


In [7]:
import akshare as ak
import pandas as pd

def get_stock_turnover_rate(stock_code, days=5):
    """获取股票近N日换手率
    
    Args:
        stock_code: 股票代码，如 '000001'（不带市场标识）
        days: 获取天数，默认5天
    
    Returns:
        包含日期和换手率的字典列表
    """
    try:
        # 获取股票历史数据，包含换手率
        df = ak.stock_zh_a_hist(
            symbol=stock_code,
            period="daily",
            adjust="qfq"  # 前复权
        )
        
        # 选取最近N天的数据
        df = df.tail(days)
        
        # 转换为列表格式返回
        result = []
        for _, row in df.iterrows():
            result.append({
                'date': row['日期'],
                'turnover_rate': round(float(row['换手率']), 2)
            })
        
        return result
    
    except Exception as e:
        print(f"获取数据出错: {e}")
        return None

# 使用示例
if __name__ == "__main__":
    stock_code = '110092'  # 以平安银行为例
    turnover_rates = get_stock_turnover_rate(stock_code)
    
    if turnover_rates:
        print(f"{stock_code} 最近5日换手率：")
        for item in turnover_rates:
            print(f"日期: {item['date']}, 换手率: {item['turnover_rate']}%")

获取数据出错: '110092'


In [10]:
import akshare as ak
import pandas as pd

df = None

def get_cb_turnover_rate(bond_code, days=5):
    """获取可转债近N日换手率
    
    Args:
        bond_code: 可转债代码，如 '123456'（不带市场标识）
        days: 获取天数，默认5天
    
    Returns:
        包含日期和换手率的字典列表
    """
    try:
        # 获取可转债历史数据
        df = ak.bond_zh_hs_cov_daily(
            symbol=bond_code
        )
        


        # 选取最近N天的数据
        df = df.head(days)  # bond_zh_hs_cov_daily 返回的数据是倒序的，最新的在前
        
        # 转换为列表格式返回
        result = []
        for _, row in df.iterrows():
            result.append({
                'date': row['日期'],
                'turnover_rate': round(float(row['换手率']), 2)
            })
        
        return result
    
    except Exception as e:
        print(f"获取数据出错: {e}")
        return None

# 使用示例
if __name__ == "__main__":
    bond_code = '110092'  # 替换为实际的可转债代码
    turnover_rates = get_cb_turnover_rate(bond_code)
    
    if turnover_rates:
        print(f"{bond_code} 最近5日换手率：")
        for item in turnover_rates:
            print(f"日期: {item['date']}, 换手率: {item['turnover_rate']}%")

获取数据出错: 'date'
